In [ ]:
from dotenv import *
import os, subprocess
load_dotenv(find_dotenv())

project_root = '..'

In [ ]:
query_path = os.path.join(project_root, os.environ['DEMO_QUERY_PATH'])

# Play the query

In [ ]:
import IPython
IPython.display.Audio(query_path)

# What song was actually queried?

In [ ]:
song_num = 13
target_query_path = str(song_num).zfill(5) + '.mid';
target_query_path

In [ ]:
midi_wav_path = os.path.join(project_root, os.environ['MIDI_WAV_DIR'], str(song_num).zfill(5) + '.wav')
IPython.display.Audio(midi_wav_path)

In [ ]:
total_songs = len(os.listdir(os.path.join(project_root, os.environ['MIDI_DIR'])))

# Measure performance

In [ ]:
%%time
from utils import get_prediction_df
res_df = get_prediction_df(query_path)

In [ ]:
import numpy as np
def get_rank(res_df, path):
    if len(res_df[res_df.midi_paths.str.contains(path)]) == 0:
        return -1
    return res_df.get_value(res_df[res_df.midi_paths.str.contains(path)].index[0], 'rank')

def get_error(res_df, path):
    if len(res_df[res_df.midi_paths.str.contains(path)]) == 0:
        return np.inf
    return res_df.get_value(res_df[res_df.midi_paths.str.contains(path)].index[0], 'error')

In [ ]:
rank = get_rank(res_df, target_query_path)
print 'Ranked song %d out of %d' % (rank, total_songs)

# Measure performance with increasing noise

In [ ]:
import librosa
y, sr = librosa.load(query_path, sr=8000)

In [ ]:
from scipy.stats import signaltonoise
import numpy as np

def snr(y):
    return (10 * np.log10(np.abs(signaltonoise(y))))

init_snr = snr(y)
print 'SNR = %0.02f dB' % init_snr

In [ ]:
import os, scipy
tmp_query_path = '.tmp_query.wav'
y_noise = y
snrs = [init_snr]
ranks = [rank]
dfs = [res_df]
errors = [get_error(res_df, target_query_path)]
iter_n = 0

while ranks[-1] != -1:
    iter_n += 1
    print iter_n
    y_noise = (y_noise + 0.2 * np.random.normal(0, 1, len(y))).astype('float16')
    
    scipy.io.wavfile.write(tmp_query_path, sr, y_noise)
    noise_df = get_prediction_df(tmp_query_path)
    noise_rank = get_rank(noise_df, target_query_path)
    
    dfs.append(noise_df)
    ranks.append(noise_rank)
    snrs.append(snr(y_noise))
    errors.append(get_error(noise_df, target_query_path))
       
os.remove(tmp_query_path)

In [ ]:
IPython.display.Audio(y_noise, rate=sr)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
y_ranks = ranks[:-1]
plt.plot(range(len(y_ranks)), y_ranks)
plt.ylabel('Rank')
plt.xlabel('Iterations of Adding Noise')
plt.title('Rank of Song vs Noise')

# Signal Comparison

In [ ]:
y_harm, y_perc = librosa.effects.hpss(y_noise)
noise_plot, = plt.plot(y_harm, color='b', label='w/ Noise')
y_harm, y_perc = librosa.effects.hpss(y)
clean_plot, = plt.plot(y_harm, color='r', label='w/o Noise')
plt.legend([noise_plot, clean_plot])